# How prompts influence LMs?

Code for analyzing and comparing LM behavior given prompts of different nature (LAMA, Autoprompt, LPAQA).

## Initialization

Import the libraries.

In [1]:
import argparse
import os
import random
import logging
import torch

from models import build_model_by_name
from utils import load_vocab, load_data, batchify, evaluate, get_relation_meta

ImportError: cannot import name 'CommitOperationAdd' from 'huggingface_hub' (/Users/corentk/miniconda3/envs/optiprompt/lib/python3.10/site-packages/huggingface_hub/__init__.py)

Intialize the logger, used to store info related to the experiment.
Also, define the init_template function (idk what's the purpose).

In [ ]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

def init_template(prompt_file, relation):
    relation = get_relation_meta(prompt_file, relation)
    return relation['template']

Handle input argument, in this notebook we are only using the default arguments.

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='bert-base-cased', help='the huggingface model name')
parser.add_argument('--output_dir', type=str, default='output', help='the output directory to store prediction results')
parser.add_argument('--common_vocab_filename', type=str, default='common_vocab_cased.txt', help='common vocabulary of models (used to filter triples)')
parser.add_argument('--prompt_file', type=str, default='prompts/LAMA_relations.jsonl', help='prompt file containing 41 relations')

parser.add_argument('--test_data_dir', type=str, default="data/filtered_LAMA")
parser.add_argument('--eval_batch_size', type=int, default=32)

parser.add_argument('--seed', type=int, default=6)
parser.add_argument('--output_predictions', default=True, help='whether to output top-k predictions')
parser.add_argument('--k', type=int, default=5, help='how many predictions will be outputted')

## Experiment

The following lines of code are used to initialize an LM.
Then the LM iterates on the evaluation data, using a specific prompt types (given as argument, see above).

In [ ]:
if __name__ == "__main__":

    # Parse arguments
    args = parser.parse_args()

    # Logger
    logger.info(args)

    # Initialize GPUs
    device=torch.device("mps") # if using Macbook M1 chip
    n_gpu = torch.cuda.device_count()
    logger.info('# GPUs: %d'%n_gpu)
    if n_gpu == 0:
        logger.warning('No GPU found! exit!')
    logger.info('Model: %s'%args.model_name)

    # Random seed
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    if torch.cuda.device_count() > 1:
        torch.cuda.manual_seed_all(args.seed)

    # Initialize the LM
    model = build_model_by_name(args)

    # Do something with the vocabulary, idk what
    if args.common_vocab_filename is not None:
        vocab_subset = load_vocab(args.common_vocab_filename)
        logger.info('Common vocab: %s, size: %d'%(args.common_vocab_filename, len(vocab_subset)))
        filter_indices, index_list = model.init_indices_for_filter_logprobs(vocab_subset)
    else:
        filter_indices = None
        index_list = None

    for relation in os.listdir(args.test_data_dir):
        relation = relation.split(".")[0]
        print("RELATION {}".format(relation))

        output_dir = os.path.join(args.output_dir, os.path.basename(args.prompt_file).split(".")[0],args.model_name.replace("/","_"))
        os.makedirs(output_dir , exist_ok=True)

        template = init_template(args.prompt_file, relation)
        logger.info('Template: %s'%template)

        test_data = os.path.join(args.test_data_dir, relation + ".jsonl")
        eval_samples = load_data(test_data, template, vocab_subset=vocab_subset, mask_token=model.MASK)
        eval_samples_batches, eval_sentences_batches = batchify(eval_samples, args.eval_batch_size * n_gpu)
        evaluate(model, eval_samples_batches, eval_sentences_batches, filter_indices, index_list, output_topk=output_dir if args.output_predictions else None)


    